In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 1차 : 쿠팡 상품 리스트 추출
- 밀키트 : 502482
- 국탕전골 : 502483
- 덮밥/비빔밥 : 502484
- 스테이크/고기 502485
- 면/파스타/감바스 502486
- 분식 502487
- 어린이 만들기 겸용 502489
- 중식요리 502490
- 기타요리 502491

In [2]:
# 쿠팡 전체 html 추출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt,
           "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

In [3]:
cg_url = 'https://www.coupang.com/np/categories/'
categoryId = '502485'
page = '1'

URL = cg_url+categoryId+"?page="+page
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

cg_soup = get_coupang_item(URL, user_agt)

# 1차: 필요 정보만 추출
- 상품id, 상품명, 정가, 할인율, 할인가, 별점, 리뷰수, 품절여부

In [4]:
cg_soup2 = cg_soup.select("#productList li")

In [5]:
item_list = ['상품id','data-item-id','data-vendor-item-id','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)
#coupang_items.info()

In [6]:
for cg_soup in cg_soup2:
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text
    price_value = re.sub(r'[^0-9]','',price_value)
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip()
        base_price = re.sub(r"^[0-9]", '', base_price)
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = 'NaN'
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame에 추가
    coupang_item = pd.DataFrame({'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                 , '상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                 , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    coupang_items = pd.concat([coupang_items,coupang_item])
coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 0
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   상품id                 36 non-null     object
 1   data-item-id         36 non-null     object
 2   data-vendor-item-id  36 non-null     object
 3   상품명                  36 non-null     object
 4   정가                   36 non-null     object
 5   할인율                  36 non-null     object
 6   판매가                  36 non-null     object
 7   100g당_가격             36 non-null     object
 8   별점                   36 non-null     object
 9   리뷰수                  36 non-null     object
 10  품절여부                 36 non-null     object
dtypes: object(11)
memory usage: 3.4+ KB


In [7]:
coupang_items.head()

,상품id,data-item-id,data-vendor-item-id,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부
0,2212937714,3767825253,71752894115,"곰곰 골든비프 찹스테이크 밀키트, 571g, 1개","6,990",5%,15990,"(100g당 2,800원)",5.0,13114,0
0,1592200730,2720528064,70710761564,"마이셰프 마이셰프 레드와인스테이크, 539g, 1개","7,900",11%,15900,"(100g당 2,950원)",4.5,3008,0
0,320547546,1025762372,5469085011,"앙트레 쿠킹박스 찹스테이크 2인분, 535g, 1개",16900,0,16900,"(100g당 3,159원)",4.5,12488,0
0,4550022981,5521392375,72820915162,"명장의 맛 시즈닝 척아이롤 스테이크, 350g, 1팩",14900,0,14900,"(100g당 4,257원)",4.5,5594,0
0,6439578446,13933872806,81183220017,"딜리조이 찹스테이크 밀키트, 1개","6,590",9%,14990,NaN,4.5,76,0


# 2차 : 구성 정보 추출
- 메뉴이름, 구성정보

In [8]:
# product_url = 'https://www.coupang.com/vp/products/'
# productId = '1866720935' #data-product-id
# itemsId = '3172927841'  #data-item-id
# vendorItemId = '71160443802' #data-vendor-item-id
# user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"
# URL = product_url+productId+'/items/'+itemsId+'/vendoritems/'+vendorItemId
# pdt_soup = get_coupang_item(URL, user_agt)

In [8]:
data_items = pd.DataFrame(columns=['상품id', '구성정보'])
#data_items.head()

In [178]:
coupang_items[['상품id','data-item-id','data-vendor-item-id']]

,상품id,data-item-id,data-vendor-item-id
0,2212937714,3767825253,71752894115
0,1592200730,2720528064,70710761564
0,320547546,1025762372,5469085011
0,4550022981,5521392375,72820915162
0,6091413115,11358338142,78634691299
0,6439578446,13933872806,81183220017
0,6223984745,12470789188,79739631592
0,6091480425,11358751325,78635100077
0,6509988958,14374414473,81618757578
0,4841176772,6256834458,73552540488


In [174]:
for productId, itemId, vendorItemId in coupang_items[['상품id','data-item-id','data-vendor-item-id']]:
    URL = product_url+productId+'/items/'+itemsId+'/vendoritems/'+vendorItemId
    pdt_soup = get_coupang_item(URL, user_agt)
    
    all = pdt_soup.find('body').children
    cnt = 0
    for item in all:
        info_of_organization = 'NaN'
        s = item.text.strip()
        if cnt == 2:
            info_of_organization = s
            cnt = 0
        
        if cnt == 1:
            cnt = 2
        
        if '구성 정보' in s:
            #print(s)
            cnt = 1
    data_item = pd.DataFrame({'상품':productId, '구성정보':info_of_organization})

ValueError: too many values to unpack (expected 3)

In [9]:
for idx in range(len(coupang_items)):
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.iloc[idx, 0] #data-product-id
    itemsId = coupang_items.iloc[idx, 1]  #data-item-id
    vendorItemId = coupang_items.iloc[idx, 2] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    
    pdt_soup = get_coupang_item(URL, user_agt)
    
    all = pdt_soup.find('body').children
    cnt = 0
    for item in all:
        s = item.text.strip()
        if cnt == 2:
            info_of_organization = s
            cnt = 3
        
        elif cnt == 1:
            cnt = 2
        
        elif '구성 정보' in s:
            #print(s)
            cnt = 1
    if not cnt==3:
        info_of_organization = 'NaN'
    data_item = pd.DataFrame({'상품id':productId, '구성정보':info_of_organization}, index=[0])
    data_items = pd.concat([data_items, data_item])
data_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    36 non-null     object
 1   구성정보    36 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


In [10]:
new_coupang_df = pd.merge(coupang_items, data_items)
new_coupang_df.head()

,상품id,data-item-id,data-vendor-item-id,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보
0,2212937714,3767825253,71752894115,"곰곰 골든비프 찹스테이크 밀키트, 571g, 1개","6,990",5%,15990,"(100g당 2,800원)",5.0,13114,0,NaN
1,1592200730,2720528064,70710761564,"마이셰프 마이셰프 레드와인스테이크, 539g, 1개","7,900",11%,15900,"(100g당 2,950원)",4.5,3008,0,"소고기, 갈릭 후레이크 튀김, 새송이버섯, 꼬마양배추, 방울토마토, 버터, 시즈닝,..."
2,320547546,1025762372,5469085011,"앙트레 쿠킹박스 찹스테이크 2인분, 535g, 1개",16900,0,16900,"(100g당 3,159원)",4.5,12488,0,"쇠고기, 마늘 편, 적양파, 피망, 파프리카, 양송이, 파슬리, 올리브오일, 스테이..."
3,4550022981,5521392375,72820915162,"명장의 맛 시즈닝 척아이롤 스테이크, 350g, 1팩",14900,0,14900,"(100g당 4,257원)",4.5,5594,0,"척아이롤 스테이크, 서류가공품(감자), 버터, 소스로 구성되어 있습니다."
4,6439578446,13933872806,81183220017,"딜리조이 찹스테이크 밀키트, 1개","6,590",9%,14990,NaN,4.5,76,0,NaN


In [12]:
new_coupang_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   상품id                 36 non-null     object
 1   data-item-id         36 non-null     object
 2   data-vendor-item-id  36 non-null     object
 3   상품명                  36 non-null     object
 4   정가                   36 non-null     object
 5   할인율                  36 non-null     object
 6   판매가                  36 non-null     object
 7   100g당_가격             36 non-null     object
 8   별점                   36 non-null     object
 9   리뷰수                  36 non-null     object
 10  품절여부                 36 non-null     object
 11  구성정보                 36 non-null     object
dtypes: object(12)
memory usage: 3.7+ KB
